This notebook contains plots and analysis statistics of FR1, including

1. [Figure 2] Accuracy/Correct Rate of FED3 session in FR1

2. [Figure 2] Food Intake Pattern in FR1

### CASK FR1 Correct Rate

In [1]:
import sys
sys.path.insert(0, '../scripts')

import os
import numpy as np
from accuracy import read_and_record, graph_group_stats, plot_cumulative_accuracy
from path import fr1_ctrl_path, fr1_cask_path, fr1_ctrl_sheets, fr1_cask_sheets
from preprocessing import read_excel_by_sheet
from intervals import perform_T_test
import warnings
warnings.filterwarnings('ignore')

import meals as ml

export_root = '../export/Figure 2'
os.makedirs(export_root, exist_ok=True)
ctrl_group_data, cask_group_data = [], []
ctrl_ending_corr, cask_ending_corr = [], []
ctrl_learned_time, cask_learned_time = [], []
ctrl_acc_dict, cask_acc_dict = {}, {}

In [2]:
for sheet in fr1_ctrl_sheets:
    data = read_and_record(path=fr1_ctrl_path, sheet=sheet, 
                        ending_corr=ctrl_ending_corr, 
                        learned_time=ctrl_learned_time,
                        acc_dict=ctrl_acc_dict)
    ctrl_group_data.append(data)

for sheet in fr1_cask_sheets:
    data = read_and_record(path=fr1_cask_path, sheet=sheet, 
                        ending_corr=cask_ending_corr, 
                        learned_time=cask_learned_time,
                        acc_dict=cask_acc_dict)
    cask_group_data.append(data)

In [ ]:
all_binned = plot_cumulative_accuracy([ctrl_group_data, cask_group_data], 
                    ["Control", 'CASK'], bin_size_sec=5,
                    export_path=os.path.join(export_root, f'cask_fr1_accuracy.svg'))

In [ ]:
all_binned[0]

#### Analysis

In [ ]:
perform_T_test(ctrl_learned_time, cask_learned_time, test_side='two-sided')

In [ ]:
graph_group_stats(ctrl=ctrl_learned_time, exp=cask_learned_time, 
                  group_names=['Ctrl', 'CASK'], stats_name='First Learned Line', unit='hours',
                  export_path=os.path.join(export_root, 'CASK_FR1_FirstLearnedLine_violinplot.svg')
                  )

In [51]:
import pickle
pickle.dump((ctrl_acc_dict, cask_acc_dict), open('../data/ctrl_fr1_acc.pkl', 'wb'))

#### Ctrl/Cask Ending/Overall Correct Rate

In [ ]:
perform_T_test(ctrl_ending_corr, cask_ending_corr, test_side='two-sided')

In [ ]:
graph_group_stats(ctrl=ctrl_ending_corr, exp=cask_ending_corr, group_names=['Ctrl', 'CASK'], stats_name='Average Correct Rate', unit='%',
                  export_path=os.path.join(export_root, 'CASK_FR1_accuracy_barplot.svg'))

### CASK FR1 Meal Analysis

In [8]:
export_root = '../export/Figure 2/CASK_FR1_Meal/'
os.makedirs(export_root, exist_ok=True)

# Initialize all data lists
ctrl_avg_pellet, cask_avg_pellet = [], []  # pellet per hour
ctrl_dark_meals, cask_dark_meals = [], []  # percentage of meals in inactive/dark phase
ctrl_fir_meal, cask_fir_meal = [], []
ctrl_fir_good_meal, cask_fir_good_meal = [], []
flip_ctrl_meal_count, flip_cask_meal_count = [], []
ctrl_in_meal_ratios, cask_in_meal_ratios = [], []

In [ ]:
for sheet in fr1_ctrl_sheets:
    metrics = ml.process_meal_data(sheet, fr1_ctrl_path, is_cask=False, 
                                   export_root=export_root)
    
    ctrl_avg_pellet.append(metrics['avg_pellet'])
    ctrl_dark_meals.append(metrics['dark_meals'])
    ctrl_fir_meal.append(metrics['fir_meal'])
    ctrl_fir_good_meal.append(metrics['fir_good_meal'])
    flip_ctrl_meal_count.append(metrics['meal_count'])
    ctrl_in_meal_ratios.append(metrics['in_meal_ratio'])

for sheet in fr1_cask_sheets:
    metrics = ml.process_meal_data(sheet, fr1_cask_path, is_cask=True, 
                                   export_root=export_root)
    
    cask_avg_pellet.append(metrics['avg_pellet'])
    cask_dark_meals.append(metrics['dark_meals'])
    cask_fir_meal.append(metrics['fir_meal'])
    cask_fir_good_meal.append(metrics['fir_good_meal'])
    flip_cask_meal_count.append(metrics['meal_count'])
    cask_in_meal_ratios.append(metrics['in_meal_ratio'])

In [10]:
ctrl_light_meals = 1 - np.array(ctrl_dark_meals)
cask_light_meals = 1 - np.array(cask_dark_meals)

#### Inactive and active meal in group

In [ ]:
perform_T_test(ctrl_light_meals, ctrl_dark_meals)

In [ ]:
perform_T_test(cask_dark_meals, cask_light_meals)

#### Average Meal Count per Day

In [ ]:
perform_T_test(flip_ctrl_meal_count, flip_cask_meal_count, test_side='two-sided')

#### Inactive and active meal between group

In [ ]:
perform_T_test(ctrl_dark_meals, cask_dark_meals, test_side='two-sided')

### Average Pellet

In [ ]:
graph_group_stats(ctrl_avg_pellet, cask_avg_pellet, stats_name='FR1 Average Pellet', unit='pellets', group_names=['Control', 'CASK'],
                  export_path=os.path.join(export_root, '../CASK_FR1_pellet count.svg'))

In [ ]:
perform_T_test(ctrl_avg_pellet, cask_avg_pellet, test_side='two-sided')

#### Time needed to start the first meal

In [ ]:
graph_group_stats(ctrl_fir_meal, cask_fir_meal, stats_name='First Meal Time', unit='hours',group_names=['Control', 'CASK'],
                  export_path=os.path.join(export_root, '../CASK_FR1_1stMealTime.svg'))

In [ ]:
perform_T_test(ctrl_fir_meal, cask_fir_meal, test_side='two-sided')

#### Time needed to start the first **Good** meal

In [ ]:
graph_group_stats(ctrl_fir_good_meal, cask_fir_good_meal, stats_name='First Good Meal Time', unit='hours', group_names=['Control', 'CASK'],
                    export_path=os.path.join(export_root, '../CASK_FR1_1stGoodMealTime.svg'))

In [ ]:
perform_T_test(ctrl_fir_good_meal, cask_fir_good_meal, test_side='two-sided')

### In-meal pellet percentage

In [ ]:
print(np.mean(ctrl_in_meal_ratios), np.std(ctrl_in_meal_ratios) / np.sqrt(len(ctrl_in_meal_ratios)))
print(np.mean(cask_in_meal_ratios), np.std(cask_in_meal_ratios) / np.sqrt(len(cask_in_meal_ratios)))